# Using Gemma 3 (4B) to identify images

<a target="_blank" href="https://colab.research.google.com/github/simonguest/CS-394/blob/main/src/04/notebooks/vlm-gemma-3-4b.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
<a target="_blank" href="https://github.com/simonguest/CS-394/raw/refs/heads/main/src/04/notebooks/vlm-gemma-3-4b.ipynb">
  <img src="https://img.shields.io/badge/Download_.ipynb-blue" alt="Download .ipynb"/>
</a>

In [ ]:
from transformers import pipeline
import torch
import os
import sys

if 'google.colab' in sys.modules:
  from google.colab import userdata # type:ignore
  os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
  print("Replicate API Token set for Colab")
else:
  load_dotenv()
  print("Loaded env vars from .env")


pipe = pipeline(
    "image-text-to-text",
    model="google/gemma-3-4b-it",
    device="cuda",
    dtype=torch.bfloat16
)

Replicate API Token set for Colab


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

...

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [2]:
import gradio as gr

def analyze(image, prompt, max_new_tokens=256, system_prompt="You are a medical expert."):
    if image is None:
        return "Please upload an image."

    if prompt is None or not prompt.strip():
        prompt = "Describe the image."

    # Chat-style messages (instruction-tuned models expect this format)
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_prompt}],
        },
        {
            "role": "user",
            "content": [
                {"type": "image"},                 # placeholder; actual PIL image passed via images=
                {"type": "text", "text": prompt},
            ],
        },
    ]

    # Run inference
    outputs = pipe(
        text=messages,
        images=[image],
        max_new_tokens=int(max_new_tokens),
        return_full_text=False,
    )

    # Gemma 3 pipeline output is typically chat-like; extract last assistant content
    out = outputs[0]["generated_text"]
    if isinstance(out, list) and len(out) > 0 and isinstance(out[-1], dict) and "content" in out[-1]:
        return out[-1]["content"]

    # Fallback in case output format differs
    return str(out)

with gr.Blocks(title="Medical Expert") as demo:
    gr.Markdown("## Gemma 3 Vision — upload an image + ask a question")

    with gr.Row():
        img = gr.Image(type="pil", label="Input image")
        with gr.Column():
            prompt = gr.Textbox(lines=4, label="Prompt", value="What is in this image?")
            btn = gr.Button("Run")

    output = gr.Textbox(lines=12, label="Model output")

    btn.click(
        fn=analyze,
        inputs=[img, prompt],
        outputs=output,
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8ef19afd3f2797edf4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
